In [1]:
import json
import pickle
import random
import math
from time import time
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, concatenate, Dense, Input, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

In [2]:
class ImageCaptionEmbeddingWithConsensus:
    def __init__(self, image_name, caption, image_embedding, caption_embedding, consensus):
        self.image_name = image_name
        self.caption = caption
        self.image_embedding = image_embedding
        self.caption_embedding = caption_embedding
        self.consensus = consensus

In [3]:
image_caption_objects = 'corrected_images_captions_with_embeddings.pkl'
mirrored_objects = 'mirrored_images_captions_with_embeddings.pkl'
aug_data = 'augmented_data_embeddings.pkl'
aug_data_mirrored = 'augmented_data_embeddings_mirrored.pkl'

# encoding = latin1: needs to be specified in order to unpickle numpy arrays from Python2 to Python3
with open(image_caption_objects, 'rb') as f1:
    data_list_1 = pickle.load(f1, encoding="latin1")
    print(len(data_list_1))
    
with open(mirrored_objects, 'rb') as f2:
    data_list_2 = pickle.load(f2, encoding='latin1')
    print(len(data_list_2))
    
with open(aug_data, 'rb') as f3:
    data_list_3 = pickle.load(f3, encoding='latin1')
    print(len(data_list_3))
    
with open(aug_data_mirrored, 'rb') as f4:
    data_list_4 = pickle.load(f4, encoding='latin1')
    print(len(data_list_4))

12039
12039
5000
5000


In [26]:
data_list = data_list_1 + data_list_2 + data_list_3 + data_list_4
print(len(data_list))

34078


In [3]:
# This cell is here to run the model with human only ratings - Don't run this unless needed
image_caption_objects = 'human_scores_only_embeddings.pkl'
mirrored_objects = 'human_scores_only_embeddings_mirrored.pkl'

with open(image_caption_objects, 'rb') as f1:
    data_list_1 = pickle.load(f1, encoding="latin1")
    print(len(data_list_1))
    
with open(mirrored_objects, 'rb') as f2:
    data_list_2 = pickle.load(f2, encoding='latin1')
    print(len(data_list_2))
    

data_list = data_list_1 + data_list_2
print(len(data_list))

12039
12039
24078


In [4]:
print(data_list[1089].consensus)

0.0


In [5]:
# To concatenate, use numpy concatenate. axis = 1 results in a numpy array [[2048]], need to access [0]
im_emb = data_list[0].image_embedding
cap_emb = data_list[0].caption_embedding
concat = np.concatenate((im_emb, cap_emb), axis = 1)

In [6]:
# May need to exclude some data - especially data with no human ratings
# Note that it seems like with no human ratings, there is still consensus.
# And if there are no cosine similiarty data, there is still consensus ('Z0', 'Z1' don't always exist)
#list(filter(lambda x: x < 0, number_list))
cleaned_data_list = list(filter(lambda obj: float(obj.consensus) > 0, data_list))
random.shuffle(cleaned_data_list)
print(len(cleaned_data_list))

21262


In [7]:
im_cap_concat_list = [np.concatenate((obj.image_embedding, obj.caption_embedding), axis=1)[0].tolist() for obj in cleaned_data_list]
im_cap_concat = np.array(im_cap_concat_list)

# Note that the consensus is rounded, then subtract 1 so that the scores are 0 - 4
labels_list = [round(float(obj.consensus)) - 1 for obj in cleaned_data_list]
labels = np.array(labels_list)

In [23]:
print('Count 1 = ', labels_list.count(0))
print('Count 2 = ', labels_list.count(1))
print('Count 3 = ', labels_list.count(2))
print('Count 4 = ', labels_list.count(3))
print('Count 5 = ', labels_list.count(4))

Count 1 =  5834
Count 2 =  3054
Count 3 =  3942
Count 4 =  6158
Count 5 =  2274


In [8]:
print(len(im_cap_concat))
print(len(labels))
print(len(im_cap_concat[0]))
print(im_cap_concat.shape)
print(im_cap_concat[0].shape)
print(im_cap_concat[0])

21262
21262
2048
(21262, 2048)
(2048,)
[-0.01290849 -0.01551739  0.00388946 ... -0.06732516 -0.00289487
 -0.01871625]


In [13]:
early_stopping_monitor = EarlyStopping(patience=3)
batch_size = 100
num_epochs = 1000
learning_rate = 0.00001
decay_rate = learning_rate / num_epochs

model_2 = Sequential()
model_2.add(Dense(512, activation='relu', input_shape=(2048,)))
model_2.add(Dropout(0.2))
model_2.add(Dense(512, activation='relu'))
model_2.add(Dropout(0.2))
model_2.add(Dense(512, activation='relu'))
model_2.add(Dropout(0.2))
#model_2.add(Dense(512, activation='relu'))
model_2.add(Dense(5, activation='softmax'))
opt = Adam(learning_rate=learning_rate, decay=decay_rate)
loss_func = SparseCategoricalCrossentropy()
tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))
# NOTE: to use categorical cross entropy, I had to subtract 1 from the consensus to make the ratings from 0 to 4
model_2.compile(optimizer=opt, loss=loss_func, metrics=['accuracy'])

In [14]:
train_size = math.floor(len(im_cap_concat) * 0.8)
print(train_size)

test_size = len(im_cap_concat) - train_size
print(test_size)

model_2.fit(im_cap_concat[:train_size], labels[:train_size], batch_size=batch_size, epochs=num_epochs, \
            validation_split=0.2, callbacks=[tensorboard])



17009
4253
Epoch 1/1000
137/137 [==============================] - 1s 7ms/step - loss: 1.5895 - accuracy: 0.2818 - val_loss: 1.5733 - val_accuracy: 0.2745
Epoch 2/1000
137/137 [==============================] - 1s 6ms/step - loss: 1.5546 - accuracy: 0.2889 - val_loss: 1.5509 - val_accuracy: 0.2745
Epoch 3/1000
137/137 [==============================] - 1s 6ms/step - loss: 1.5419 - accuracy: 0.3003 - val_loss: 1.5474 - val_accuracy: 0.2898
Epoch 4/1000
137/137 [==============================] - 1s 6ms/step - loss: 1.5388 - accuracy: 0.3095 - val_loss: 1.5445 - val_accuracy: 0.3233
Epoch 5/1000
137/137 [==============================] - 1s 6ms/step - loss: 1.5359 - accuracy: 0.3181 - val_loss: 1.5412 - val_accuracy: 0.3325
Epoch 6/1000
137/137 [==============================] - 1s 6ms/step - loss: 1.5310 - accuracy: 0.3245 - val_loss: 1.5366 - val_accuracy: 0.3369
Epoch 7/1000
137/137 [==============================] - 1s 6ms/step - loss: 1.5259 - accuracy: 0.3375 - val_loss: 1.5310 - va

In [15]:
predictions = model_2.predict(im_cap_concat[train_size:])

In [16]:
test_loss, test_accuracy = model_2.evaluate(im_cap_concat[train_size:], labels[train_size:])

133/133 [==============================] - 0s 2ms/step - loss: 1.9652 - accuracy: 0.6950


In [17]:
train_loss, train_accuracy = model_2.evaluate(im_cap_concat[:train_size], labels[:train_size])

532/532 [==============================] - 1s 2ms/step - loss: 0.4323 - accuracy: 0.9301


In [18]:
print(predictions[0])

predicted_ratings = []
for pred in predictions:
    predicted_ratings.append(np.argmax(pred)+1)

print(len(predicted_ratings))

# Need to add back the 1 that was subtracted for training
actual_ratings = [num + 1 for num in labels[train_size:]]
print(len(actual_ratings))

[3.3485245e-05 9.9110717e-01 8.7410565e-03 1.1789031e-04 4.4480328e-07]
4253
4253


In [19]:
def find_accuracy(predicted, actual):
    totals_by_rating = [0, 0, 0, 0, 0, 0]
    correct_by_rating = [0, 0, 0, 0, 0, 0]
    num_correct = 0
    num_ratings = 0
    
    for pred, act in zip(predicted, actual):
        totals_by_rating[act] += 1
        if pred == act:
            num_correct += 1
            correct_by_rating[act] += 1
        num_ratings += 1
        
    print('Accuracy = ', num_correct / num_ratings)
    
    print('---------------------------')
    print('Accuracy by rating 1 - 5')
    for idx, (correct, total) in enumerate(zip(correct_by_rating, totals_by_rating)):
        if idx == 0:
            continue
        print('Rating = ', idx, '| Accuracy = ', correct/total)
    
    
find_accuracy(predicted_ratings, actual_ratings)

Accuracy =  0.6950387961438984
---------------------------
Accuracy by rating 1 - 5
Rating =  1 | Accuracy =  0.8048780487804879
Rating =  2 | Accuracy =  0.5820379965457686
Rating =  3 | Accuracy =  0.6317169069462647
Rating =  4 | Accuracy =  0.7088122605363985
Rating =  5 | Accuracy =  0.62882096069869


In [20]:
print(predicted_ratings)

[2, 4, 4, 1, 1, 1, 2, 3, 1, 2, 4, 1, 5, 2, 4, 3, 1, 4, 1, 4, 2, 1, 4, 1, 1, 4, 1, 4, 4, 1, 4, 4, 1, 3, 5, 1, 1, 1, 3, 2, 5, 2, 5, 4, 5, 1, 3, 1, 4, 4, 4, 4, 2, 4, 3, 4, 4, 1, 4, 4, 4, 5, 4, 3, 5, 1, 4, 1, 3, 4, 1, 4, 3, 1, 3, 4, 4, 1, 4, 4, 3, 2, 4, 1, 1, 3, 1, 5, 3, 4, 1, 4, 1, 5, 1, 1, 2, 1, 1, 3, 3, 4, 4, 5, 2, 1, 1, 4, 1, 2, 3, 3, 1, 4, 4, 2, 5, 4, 4, 1, 4, 3, 5, 2, 4, 4, 4, 4, 2, 1, 4, 4, 4, 1, 2, 4, 4, 2, 4, 4, 4, 2, 2, 3, 5, 4, 4, 2, 4, 2, 3, 3, 4, 4, 4, 5, 2, 5, 4, 4, 5, 4, 1, 1, 4, 3, 3, 3, 1, 1, 2, 2, 4, 2, 3, 1, 4, 3, 1, 2, 4, 1, 5, 4, 3, 1, 1, 1, 3, 1, 1, 4, 3, 3, 1, 3, 1, 2, 4, 2, 1, 5, 4, 3, 1, 3, 5, 3, 3, 4, 2, 1, 1, 5, 3, 4, 4, 4, 2, 1, 4, 4, 4, 4, 3, 4, 1, 1, 5, 4, 5, 1, 1, 1, 3, 1, 3, 2, 2, 2, 1, 4, 1, 2, 5, 3, 4, 4, 4, 4, 1, 5, 4, 1, 5, 1, 1, 5, 1, 5, 5, 4, 1, 1, 5, 4, 3, 1, 1, 1, 1, 3, 2, 3, 1, 5, 3, 1, 2, 4, 5, 4, 1, 3, 5, 1, 1, 1, 1, 4, 1, 5, 1, 4, 3, 4, 1, 1, 2, 2, 4, 3, 1, 5, 4, 3, 1, 4, 3, 1, 3, 1, 1, 2, 3, 2, 1, 5, 4, 1, 1, 5, 1, 4, 4, 1, 4, 1, 4, 4, 2, 3, 1, 

Using keras tuner for hyperaparameter tuning

In [ ]:
#!pip install -U keras-tuner

In [41]:
import kerastuner as kt

def build_model(hp):
    inputs = tf.keras.Input(shape=(2048,))
    x = inputs
    for _ in range(3):
        x = tf.keras.layers.Dense(512, activation='relu')(x)
        x = tf.keras.layers.Dropout(hp.Float('dropout', 0, 0.3, step=0.05, default=0.2))(x)
    
    outputs = tf.keras.layers.Dense(5, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(hp.Float('learning_rate', 0.00001, 0.00002, step=0.000005)),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    tensorboard = TensorBoard(log_dir='logs/{}'.format(time()))
    return model
                
                
#tuner = kt.Hyperband(build_model, objective='val_accuracy', max_epochs=300, hyperband_iterations=2, directory='augmented_3layer')

tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=50, directory='random_search_sept_20')

tuner.search(im_cap_concat[:train_size], labels[:train_size], epochs=300, validation_split=0.2, callbacks=[EarlyStopping('val_loss', patience=3)])


Epoch 1/300
682/682 [==============================] - ETA: 0s - loss: 1.6167 - accuracy: 0.15 - ETA: 2s - loss: 1.6119 - accuracy: 0.16 - ETA: 2s - loss: 1.6118 - accuracy: 0.16 - ETA: 2s - loss: 1.6099 - accuracy: 0.19 - ETA: 2s - loss: 1.6086 - accuracy: 0.21 - ETA: 2s - loss: 1.6062 - accuracy: 0.24 - ETA: 2s - loss: 1.6046 - accuracy: 0.26 - ETA: 2s - loss: 1.6027 - accuracy: 0.27 - ETA: 2s - loss: 1.6003 - accuracy: 0.28 - ETA: 2s - loss: 1.5986 - accuracy: 0.28 - ETA: 2s - loss: 1.5959 - accuracy: 0.29 - ETA: 2s - loss: 1.5951 - accuracy: 0.29 - ETA: 2s - loss: 1.5936 - accuracy: 0.29 - ETA: 2s - loss: 1.5911 - accuracy: 0.30 - ETA: 2s - loss: 1.5896 - accuracy: 0.30 - ETA: 1s - loss: 1.5879 - accuracy: 0.30 - ETA: 1s - loss: 1.5876 - accuracy: 0.30 - ETA: 1s - loss: 1.5861 - accuracy: 0.30 - ETA: 1s - loss: 1.5843 - accuracy: 0.30 - ETA: 1s - loss: 1.5818 - accuracy: 0.30 - ETA: 1s - loss: 1.5810 - accuracy: 0.30 - ETA: 1s - loss: 1.5801 - accuracy: 0.30 - ETA: 1s - loss: 1.578

Epoch 1/300
682/682 [==============================] - ETA: 0s - loss: 1.6208 - accuracy: 0.06 - ETA: 2s - loss: 1.6123 - accuracy: 0.16 - ETA: 2s - loss: 1.6085 - accuracy: 0.21 - ETA: 2s - loss: 1.6067 - accuracy: 0.24 - ETA: 2s - loss: 1.6044 - accuracy: 0.26 - ETA: 2s - loss: 1.6026 - accuracy: 0.28 - ETA: 2s - loss: 1.6019 - accuracy: 0.28 - ETA: 2s - loss: 1.6001 - accuracy: 0.29 - ETA: 2s - loss: 1.5978 - accuracy: 0.29 - ETA: 2s - loss: 1.5952 - accuracy: 0.30 - ETA: 2s - loss: 1.5939 - accuracy: 0.30 - ETA: 2s - loss: 1.5923 - accuracy: 0.30 - ETA: 2s - loss: 1.5909 - accuracy: 0.30 - ETA: 2s - loss: 1.5894 - accuracy: 0.31 - ETA: 1s - loss: 1.5872 - accuracy: 0.31 - ETA: 1s - loss: 1.5856 - accuracy: 0.31 - ETA: 1s - loss: 1.5840 - accuracy: 0.31 - ETA: 1s - loss: 1.5817 - accuracy: 0.31 - ETA: 1s - loss: 1.5816 - accuracy: 0.31 - ETA: 1s - loss: 1.5804 - accuracy: 0.31 - ETA: 1s - loss: 1.5784 - accuracy: 0.31 - ETA: 1s - loss: 1.5772 - accuracy: 0.32 - ETA: 1s - loss: 1.576

Epoch 1/300
682/682 [==============================] - ETA: 0s - loss: 1.6089 - accuracy: 0.15 - ETA: 2s - loss: 1.6063 - accuracy: 0.27 - ETA: 2s - loss: 1.6045 - accuracy: 0.30 - ETA: 2s - loss: 1.6028 - accuracy: 0.31 - ETA: 2s - loss: 1.6006 - accuracy: 0.32 - ETA: 2s - loss: 1.5992 - accuracy: 0.33 - ETA: 2s - loss: 1.5980 - accuracy: 0.32 - ETA: 2s - loss: 1.5974 - accuracy: 0.32 - ETA: 2s - loss: 1.5968 - accuracy: 0.32 - ETA: 2s - loss: 1.5951 - accuracy: 0.32 - ETA: 2s - loss: 1.5935 - accuracy: 0.32 - ETA: 2s - loss: 1.5901 - accuracy: 0.33 - ETA: 2s - loss: 1.5883 - accuracy: 0.33 - ETA: 2s - loss: 1.5868 - accuracy: 0.33 - ETA: 1s - loss: 1.5846 - accuracy: 0.33 - ETA: 1s - loss: 1.5821 - accuracy: 0.34 - ETA: 1s - loss: 1.5805 - accuracy: 0.34 - ETA: 1s - loss: 1.5801 - accuracy: 0.33 - ETA: 1s - loss: 1.5794 - accuracy: 0.33 - ETA: 1s - loss: 1.5784 - accuracy: 0.33 - ETA: 1s - loss: 1.5779 - accuracy: 0.33 - ETA: 1s - loss: 1.5773 - accuracy: 0.33 - ETA: 1s - loss: 1.576

Epoch 1/300
682/682 [==============================] - ETA: 0s - loss: 1.6089 - accuracy: 0.15 - ETA: 2s - loss: 1.6075 - accuracy: 0.20 - ETA: 2s - loss: 1.6073 - accuracy: 0.20 - ETA: 2s - loss: 1.6061 - accuracy: 0.21 - ETA: 2s - loss: 1.6047 - accuracy: 0.23 - ETA: 2s - loss: 1.6036 - accuracy: 0.25 - ETA: 2s - loss: 1.6028 - accuracy: 0.26 - ETA: 2s - loss: 1.6012 - accuracy: 0.27 - ETA: 2s - loss: 1.6002 - accuracy: 0.28 - ETA: 2s - loss: 1.5992 - accuracy: 0.28 - ETA: 2s - loss: 1.5977 - accuracy: 0.29 - ETA: 2s - loss: 1.5967 - accuracy: 0.29 - ETA: 2s - loss: 1.5953 - accuracy: 0.30 - ETA: 1s - loss: 1.5943 - accuracy: 0.30 - ETA: 1s - loss: 1.5930 - accuracy: 0.30 - ETA: 1s - loss: 1.5916 - accuracy: 0.30 - ETA: 1s - loss: 1.5905 - accuracy: 0.30 - ETA: 1s - loss: 1.5905 - accuracy: 0.30 - ETA: 1s - loss: 1.5893 - accuracy: 0.30 - ETA: 1s - loss: 1.5884 - accuracy: 0.30 - ETA: 1s - loss: 1.5863 - accuracy: 0.31 - ETA: 1s - loss: 1.5849 - accuracy: 0.31 - ETA: 1s - loss: 1.583

Epoch 1/300
682/682 [==============================] - ETA: 0s - loss: 1.6059 - accuracy: 0.21 - ETA: 2s - loss: 1.6087 - accuracy: 0.18 - ETA: 2s - loss: 1.6073 - accuracy: 0.20 - ETA: 2s - loss: 1.6059 - accuracy: 0.22 - ETA: 2s - loss: 1.6057 - accuracy: 0.24 - ETA: 2s - loss: 1.6037 - accuracy: 0.26 - ETA: 2s - loss: 1.6033 - accuracy: 0.26 - ETA: 2s - loss: 1.6017 - accuracy: 0.27 - ETA: 2s - loss: 1.5998 - accuracy: 0.28 - ETA: 2s - loss: 1.5979 - accuracy: 0.29 - ETA: 2s - loss: 1.5964 - accuracy: 0.30 - ETA: 2s - loss: 1.5946 - accuracy: 0.30 - ETA: 2s - loss: 1.5932 - accuracy: 0.30 - ETA: 2s - loss: 1.5920 - accuracy: 0.30 - ETA: 2s - loss: 1.5913 - accuracy: 0.30 - ETA: 1s - loss: 1.5893 - accuracy: 0.30 - ETA: 1s - loss: 1.5880 - accuracy: 0.31 - ETA: 1s - loss: 1.5869 - accuracy: 0.31 - ETA: 1s - loss: 1.5864 - accuracy: 0.31 - ETA: 1s - loss: 1.5849 - accuracy: 0.31 - ETA: 1s - loss: 1.5838 - accuracy: 0.31 - ETA: 1s - loss: 1.5812 - accuracy: 0.31 - ETA: 1s - loss: 1.580

...

Epoch 1/300
682/682 [==============================] - ETA: 0s - loss: 1.6139 - accuracy: 0.06 - ETA: 2s - loss: 1.6089 - accuracy: 0.19 - ETA: 2s - loss: 1.6046 - accuracy: 0.27 - ETA: 2s - loss: 1.6018 - accuracy: 0.28 - ETA: 2s - loss: 1.5981 - accuracy: 0.30 - ETA: 2s - loss: 1.5956 - accuracy: 0.30 - ETA: 2s - loss: 1.5933 - accuracy: 0.30 - ETA: 2s - loss: 1.5911 - accuracy: 0.30 - ETA: 2s - loss: 1.5879 - accuracy: 0.31 - ETA: 2s - loss: 1.5863 - accuracy: 0.31 - ETA: 2s - loss: 1.5839 - accuracy: 0.31 - ETA: 2s - loss: 1.5826 - accuracy: 0.31 - ETA: 2s - loss: 1.5783 - accuracy: 0.31 - ETA: 1s - loss: 1.5752 - accuracy: 0.32 - ETA: 1s - loss: 1.5737 - accuracy: 0.32 - ETA: 1s - loss: 1.5714 - accuracy: 0.32 - ETA: 1s - loss: 1.5712 - accuracy: 0.32 - ETA: 1s - loss: 1.5695 - accuracy: 0.32 - ETA: 1s - loss: 1.5700 - accuracy: 0.31 - ETA: 1s - loss: 1.5679 - accuracy: 0.32 - ETA: 1s - loss: 1.5678 - accuracy: 0.32 - ETA: 1s - loss: 1.5677 - accuracy: 0.31 - ETA: 1s - loss: 1.567

Epoch 1/300
682/682 [==============================] - ETA: 0s - loss: 1.6136 - accuracy: 0.09 - ETA: 2s - loss: 1.6050 - accuracy: 0.26 - ETA: 2s - loss: 1.6019 - accuracy: 0.31 - ETA: 2s - loss: 1.6005 - accuracy: 0.31 - ETA: 2s - loss: 1.5986 - accuracy: 0.31 - ETA: 2s - loss: 1.5967 - accuracy: 0.31 - ETA: 2s - loss: 1.5934 - accuracy: 0.32 - ETA: 2s - loss: 1.5936 - accuracy: 0.31 - ETA: 2s - loss: 1.5910 - accuracy: 0.31 - ETA: 2s - loss: 1.5890 - accuracy: 0.31 - ETA: 2s - loss: 1.5872 - accuracy: 0.31 - ETA: 2s - loss: 1.5874 - accuracy: 0.31 - ETA: 2s - loss: 1.5853 - accuracy: 0.31 - ETA: 1s - loss: 1.5833 - accuracy: 0.31 - ETA: 1s - loss: 1.5822 - accuracy: 0.31 - ETA: 1s - loss: 1.5823 - accuracy: 0.31 - ETA: 1s - loss: 1.5805 - accuracy: 0.31 - ETA: 1s - loss: 1.5779 - accuracy: 0.31 - ETA: 1s - loss: 1.5766 - accuracy: 0.31 - ETA: 1s - loss: 1.5753 - accuracy: 0.31 - ETA: 1s - loss: 1.5737 - accuracy: 0.31 - ETA: 1s - loss: 1.5726 - accuracy: 0.31 - ETA: 1s - loss: 1.571

Epoch 1/300
682/682 [==============================] - ETA: 0s - loss: 1.6028 - accuracy: 0.34 - ETA: 2s - loss: 1.5998 - accuracy: 0.35 - ETA: 2s - loss: 1.5969 - accuracy: 0.34 - ETA: 2s - loss: 1.5951 - accuracy: 0.33 - ETA: 2s - loss: 1.5913 - accuracy: 0.33 - ETA: 2s - loss: 1.5881 - accuracy: 0.33 - ETA: 2s - loss: 1.5868 - accuracy: 0.32 - ETA: 2s - loss: 1.5829 - accuracy: 0.33 - ETA: 2s - loss: 1.5816 - accuracy: 0.32 - ETA: 2s - loss: 1.5796 - accuracy: 0.32 - ETA: 2s - loss: 1.5761 - accuracy: 0.32 - ETA: 2s - loss: 1.5763 - accuracy: 0.32 - ETA: 2s - loss: 1.5732 - accuracy: 0.32 - ETA: 2s - loss: 1.5725 - accuracy: 0.32 - ETA: 1s - loss: 1.5715 - accuracy: 0.32 - ETA: 1s - loss: 1.5673 - accuracy: 0.32 - ETA: 1s - loss: 1.5668 - accuracy: 0.32 - ETA: 1s - loss: 1.5672 - accuracy: 0.32 - ETA: 1s - loss: 1.5647 - accuracy: 0.32 - ETA: 1s - loss: 1.5642 - accuracy: 0.32 - ETA: 1s - loss: 1.5642 - accuracy: 0.32 - ETA: 1s - loss: 1.5617 - accuracy: 0.33 - ETA: 1s - loss: 1.561

Epoch 1/300
682/682 [==============================] - ETA: 0s - loss: 1.6158 - accuracy: 0.12 - ETA: 2s - loss: 1.6098 - accuracy: 0.19 - ETA: 2s - loss: 1.6091 - accuracy: 0.20 - ETA: 2s - loss: 1.6076 - accuracy: 0.22 - ETA: 2s - loss: 1.6073 - accuracy: 0.23 - ETA: 2s - loss: 1.6069 - accuracy: 0.24 - ETA: 2s - loss: 1.6064 - accuracy: 0.24 - ETA: 2s - loss: 1.6053 - accuracy: 0.26 - ETA: 2s - loss: 1.6046 - accuracy: 0.27 - ETA: 2s - loss: 1.6038 - accuracy: 0.27 - ETA: 2s - loss: 1.6024 - accuracy: 0.28 - ETA: 2s - loss: 1.6014 - accuracy: 0.29 - ETA: 2s - loss: 1.6003 - accuracy: 0.29 - ETA: 2s - loss: 1.5988 - accuracy: 0.30 - ETA: 1s - loss: 1.5984 - accuracy: 0.30 - ETA: 1s - loss: 1.5977 - accuracy: 0.30 - ETA: 1s - loss: 1.5967 - accuracy: 0.30 - ETA: 1s - loss: 1.5958 - accuracy: 0.30 - ETA: 1s - loss: 1.5949 - accuracy: 0.30 - ETA: 1s - loss: 1.5938 - accuracy: 0.30 - ETA: 1s - loss: 1.5927 - accuracy: 0.31 - ETA: 1s - loss: 1.5918 - accuracy: 0.31 - ETA: 1s - loss: 1.591

INFO:tensorflow:Oracle triggered exit


In [42]:
best_model = tuner.get_best_models(1)[0]

In [43]:
best_hyperparameters = tuner.get_best_hyperparameters(1)[0]
print(best_hyperparameters.values)

{'dropout': 0.25, 'learning_rate': 1e-05}


In [44]:
tuner.get_best_models()[0].summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 2048)]            0         
_________________________________________________________________
dense (Dense)                (None, 512)               1049088   
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 512)               262656    
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)              

In [45]:
tuner.results_summary()